In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. 데이터 불러오기
df = pd.read_csv("2. 조선_2024년_뉴스.csv")
df.columns = df.columns.str.strip()
df['뉴스제목'] = df['뉴스제목'].fillna("")

# 2. TF-IDF 변환
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(df['뉴스제목'])
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=feature_names)

# 3. 키워드 그룹 정의
keyword_groups = {
    '수주': ['수주', '계약', '납품', '공급', '수출'],
    '실적': ['실적', '이익', '매출', '흑자', '적자'],
    '미국': ['미국', '美', '연준', '금리', '인플레이션']
}

# 4. 그룹별 TF-IDF 점수 생성
for group_name, keywords in keyword_groups.items():
    tfidf_cols = [word for word in keywords if word in tfidf_df.columns]
    if tfidf_cols:
        df[f'tfidf_{group_name}'] = tfidf_df[tfidf_cols].sum(axis=1)
    else:
        df[f'tfidf_{group_name}'] = 0.0

# 5. 날짜 + 기업 기준 집계
df['날짜'] = pd.to_datetime(df['날짜'])
agg = df.groupby(['날짜', '기업명'])[[f'tfidf_{key}' for key in keyword_groups]].mean().reset_index()

# 6. 저장
agg.to_csv("5. 뉴스_TFIDF_분석포함.csv", index=False, encoding='utf-8-sig')
print("✅ TF-IDF 키워드 그룹 점수 생성 완료: 5. 뉴스_TFIDF_분석포함.csv")


✅ TF-IDF 키워드 그룹 점수 생성 완료: 5. 뉴스_TFIDF_분석포함.csv
